In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
# !pip install psycopg2-binary
import psycopg2

In [2]:
# create a database connection
sqluser = 'asem'
dbname = 'mimiciv'
hostname = 'localhost'
password='qwerasdf'
# hosp_schema_name = 'mimiciv_hosp'

# Connect to local postgres version of mimic
con = psycopg2.connect(dbname=dbname, user=sqluser, host=hostname, password=password)

In [3]:
query = \
"""
SELECT
      pat.subject_id
    , adm.hadm_id
    , icu.stay_id
    , DENSE_RANK() OVER subject_window AS hosp_stay_num
    , DENSE_RANK() OVER hosp_window AS icu_stay_num
    , DENSE_RANK() OVER icu_window AS aki_timestamp_index
    , MIN(aki_stage) OVER subject_window as current_subject_min_aki
    , MAX(aki_stage) OVER subject_window as current_subject_max_aki
    , MIN(aki_stage) OVER hosp_window as current_hosp_min_aki
    , MAX(aki_stage) OVER hosp_window as current_hosp_max_aki
    , MIN(aki_stage) OVER icu_window as current_icu_min_aki
    , MAX(aki_stage) OVER icu_window as current_icu_max_aki
    , agg_kdigo_subject.min_aki as subject_min_aki
    , agg_kdigo_subject.max_aki as subject_max_aki
    , agg_kdigo_hosp.min_aki as hosp_min_aki
    , agg_kdigo_hosp.max_aki as hosp_max_aki
    , agg_kdigo_icu.min_aki as icu_min_aki
    , agg_kdigo_icu.max_aki as icu_max_aki
    , kdigo.aki_stage_smoothed AS aki_stage
    , kdigo.charttime AS aki_time
    , CASE
        WHEN FIRST_VALUE(adm.hadm_id) OVER subject_window = adm.hadm_id THEN 1
        ELSE 0
      END AS pat_count
    , pat.anchor_age + (EXTRACT(YEAR FROM adm.admittime) - pat.anchor_year) AS age
    , pat.gender
    , adm.insurance
    , mimiciv_derived.DATETIME_DIFF(adm.dischtime, adm.admittime, 'HOUR') / 24 AS hosp_los
    , icu.los as icu_los
    , pat.dod
    , mimiciv_derived.DATETIME_DIFF(pat.dod, CAST(adm.dischtime AS DATE), 'DAY') AS days_to_death
    -- mortality flags
    , CASE WHEN mimiciv_derived.DATETIME_DIFF(pat.dod, CAST(adm.dischtime AS DATE), 'DAY') = 0 THEN 1 ELSE 0 END AS hospital_mortality
FROM mimiciv_hosp.patients pat

INNER JOIN mimiciv_hosp.admissions adm
    ON pat.subject_id = adm.subject_id

INNER JOIN mimiciv_icu.icustays icu
    ON adm.hadm_id = icu.hadm_id

INNER JOIN mimiciv_derived.kdigo_stages kdigo
    ON icu.stay_id = kdigo.stay_id

INNER JOIN (SELECT MIN(aki_stage_smoothed) AS min_aki
                , MAX(aki_stage_smoothed) AS max_aki
                , stay_id
           FROM mimiciv_derived.kdigo_stages kdigo GROUP BY kdigo.stay_id) agg_kdigo_icu
    ON icu.stay_id = agg_kdigo_icu.stay_id

INNER JOIN (SELECT MIN(aki_stage_smoothed) AS min_aki
                , MAX(aki_stage_smoothed) AS max_aki
                , hadm_id
           FROM mimiciv_derived.kdigo_stages kdigo GROUP BY kdigo.hadm_id) agg_kdigo_hosp
    ON adm.hadm_id = agg_kdigo_hosp.hadm_id

INNER JOIN (SELECT MIN(aki_stage_smoothed) AS min_aki
                , MAX(aki_stage_smoothed) AS max_aki
                , subject_id
           FROM mimiciv_derived.kdigo_stages kdigo GROUP BY kdigo.subject_id) agg_kdigo_subject
    ON pat.subject_id = agg_kdigo_subject.subject_id

WINDOW subject_window AS (PARTITION BY pat.subject_id ORDER BY adm.admittime)
    , hosp_window AS (PARTITION BY adm.hadm_id ORDER BY icu.intime)
    , icu_window AS (PARTITION BY icu.stay_id ORDER BY kdigo.charttime )

ORDER BY subject_id, hosp_stay_num, icu_stay_num, aki_timestamp_index
"""

kdigo = pd.read_sql_query(query,con)

In [4]:
query_icu = \
"""
SELECT
      MIN(pat.subject_id) AS subject_id
    , MIN(adm.hadm_id) AS hadm_id
    , icu.stay_id
    , icu.intime
    , DENSE_RANK() OVER subject_window AS hosp_stay_num
    , DENSE_RANK() OVER hosp_window AS icu_stay_num
    , MIN(kdigo.aki_stage_smoothed) as icu_min_aki
    , MAX(kdigo.aki_stage_smoothed) as icu_max_aki
    , CASE
        WHEN FIRST_VALUE(icu.stay_id) OVER subject_window = icu.stay_id THEN 1
        ELSE 0
      END AS pat_count
    , MIN(pat.anchor_age) + (EXTRACT(YEAR FROM MIN(adm.admittime)) - MIN(pat.anchor_year)) AS age
    , MIN(pat.gender) as gender
    , MIN(adm.insurance) as insurance
    , icu.los as icu_los
    , MIN(pat.dod) as dod
    -- mortality flags
    , CASE WHEN mimiciv_derived.DATETIME_DIFF(MIN(pat.dod), CAST(icu.outtime AS DATE), 'DAY') = 0 THEN 1 ELSE 0 END AS icu_mortality
FROM mimiciv_hosp.patients pat

INNER JOIN mimiciv_hosp.admissions adm
    ON pat.subject_id = adm.subject_id

INNER JOIN mimiciv_icu.icustays icu
    ON adm.hadm_id = icu.hadm_id

INNER JOIN mimiciv_derived.kdigo_stages kdigo
    ON icu.stay_id = kdigo.stay_id

GROUP BY icu.stay_id

WINDOW subject_window AS (PARTITION BY icu.subject_id ORDER BY icu.intime)
    , hosp_window AS (PARTITION BY icu.hadm_id ORDER BY icu.intime)

ORDER BY subject_id, hosp_stay_num, icu_stay_num
"""

kdigo_by_icu = pd.read_sql_query(query_icu,con)



In [5]:
kdigo

In [6]:
kdigo['subject_aki_range'] = list(map(str, zip(kdigo['subject_min_aki'], kdigo['subject_max_aki'])))
kdigo['icu_aki_range'] = list(map(str, zip(kdigo['icu_min_aki'], kdigo['icu_max_aki'])))

kdigo['subject_aki_range'] , kdigo['icu_aki_range']

In [7]:
kdigo['subject_aki_range'].hist()

In [8]:
kdigo['icu_aki_range'].hist()

In [9]:
kdigo_by_icu
kdigo_by_icu['aki_range'] = list(map(str, zip(kdigo_by_icu['icu_min_aki'], kdigo_by_icu['icu_max_aki'])))
kdigo_by_icu['icu_count'] = 1

kdigo_by_icu['aki_range'].hist()


In [10]:
kdigo_by_icu

In [11]:
from tableone import TableOne


In [12]:
[f'({i}, {j})' for i in range(4) for j in range(i, 4)]

In [13]:


columns = [
    "pat_count",
    "icu_count",
    "age", "gender", "insurance",
    'icu_mortality',
    "icu_los",
    "aki_range"
]

categorical = [
    "pat_count",
    "icu_count", 
    "gender", "insurance",
    'icu_mortality',
    "aki_range"
]

order = {
    "pat_count": [1, 0],
#     "icu_count": [1, 0],
    "gender": ["F", "M"],
    "icu_mortality": [1, 0],
    "aki_range": [f'({i}, {j})' for i in range(4) for j in range(i, 4)]
}

limit = {
    "pat_count": 1,
#     "icu_count": 1,
    "gender": 1,
    "icu_mortality": 1,
}

rename = {
    "pat_count": "Distinct patients", 
    "icu_count": "Distinct ICU Admissions",
    "age": "Age", "gender": "Administrative Gender", "insurance": "Insurance",
    "icu_los": "ICU length of stay", 
    "icu_mortality": "In-ICU mortality",
    "aki_range": "ICU AKI Range (min, max)"
}

print('ICU demographics')
icu_table = TableOne(kdigo_by_icu, columns=columns, categorical=categorical, order=order, limit=limit, rename=rename)
display(icu_table)